# Verification of Quality Check

<!-- - @author: [Hamid Ali Syed](https://syedha.com)
 -->
- Date: Jan 21, 2023

## Packages:
- pydda is for Dual/Multi-Doppler Analysis
- pyart is for reading and plotting radar data
- wradlib is also a radar toolkit like pyart, and it supports `xarray.dataset` type
- Xarray makes working with labelled `multi-dimensional` arrays in Python simple, efficient, and fun!
- Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool for working with `tabular` data
- cartopy is used to plot `maps`, `projections` and much more.
 

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pyart
# import pydda
import wradlib as wrl
import xarray as xr
import xradar as xd
import datatree as xt
# import pandas as pd
# import datetime as dt
import numpy as np
# import scipy
# import wradlib as wrl
# import netCDF4 as nc
from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter, FuncFormatter
from matplotlib import axes
import cartopy.crs as ccrs
import cartopy.feature as feat
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import glob, os, sys, re
%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [2]:
def add_map(ax, b = 0, l = 0):
    gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.3, color='black', alpha=0.3, 
                         linestyle='-', draw_labels=True)
    gl.xlabels_top = False
    gl.xlabels_bottom = b
    gl.ylabels_left = l
    gl.ylabels_right= False
    gl.xlines = True
    gl.ylines = True
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    ax.add_feature(feat.BORDERS, lw = 0.5)
#     ax.add_feature(feat.LAND, lw = 0.3, fc = [0.9,0.9,0.9])
    ax.add_feature(feat.COASTLINE, lw = 0.5)
#     ax.add_feature(feat.OCEAN, alpha = 0.5)
    ax.add_feature(feat.STATES.with_scale("10m"), alpha = 0.5, lw = 0.5, ls = "--")

In [3]:
basedir = "/depot/dawson29/data/Projects/PERiLS/obsdata/2022/Illinois_Mobile_Radar/IOP1/"
radar_type = ["COW1", "DOW7", "DOW8"]
data_folder = "QCed"

In [4]:
cow1_files = glob.glob(f"{basedir}{radar_type[0]}{os.sep}{data_folder}{os.sep}*.nc")

In [5]:
savedir = f"{basedir}{'plots'}{os.sep}"

In [36]:
for cfile in cow1_files[100:101]:
    ds = xr.open_dataset(cfile, engine="netcdf4")

In [45]:
sweeps = xd.io.backends.cfradial1._get_sweep_groups(
    ds, optional=False, site_coords=True, first_dim="time"
)
with xr.set_options(display_expand_data_vars=True):
    display(sweeps["sweep_0"])

<xarray.Dataset>
Dimensions:            (time: 720, range: 1180)
Coordinates:
  * time               (time) datetime64[ns] 2022-03-22T23:01:45.249000072 .....
  * range              (range) float32 87.5 162.4 237.4 ... 8.837e+04 8.844e+04
    azimuth            (time) float32 33.3 33.8 34.3 34.8 ... 31.8 32.3 32.8
    elevation          (time) float32 4.432 4.377 4.322 ... 0.7843 0.8172 0.8502
    latitude           (time) float64 ...
    longitude          (time) float64 ...
    altitude           (time) float64 ...
Data variables: (12/22)
    NCP                (time, range) float32 ...
    TRIP_FLA           (time, range) float32 ...
    SNRHC              (time, range) float32 ...
    SNRVC              (time, range) float32 ...
    DBZHC              (time, range) float32 ...
    DBZHC_F            (time, range) float32 ...
    ...                 ...
    DBZH               (time, range) float32 ...
    sweep_number       float64 ...
    sweep_fixed_angle  float32 ...
    sweep_mode         |S32 ...
    follow_mode        |S32 ...
    prt_mode           |S32 ...

In [88]:
sweeps['sweep_0']['longitude'] = sweeps['sweep_0'].longitude[0]
sweeps['sweep_0']['latitude'] = sweeps['sweep_0'].latitude[0]
sweeps['sweep_0']['altitude'] = sweeps['sweep_0'].altitude[0]

In [90]:
sweeps['sweep_0'].pipe(wrl.georef.georeference_dataset)

<xarray.Dataset>
Dimensions:            (time: 720, range: 1180)
Coordinates: (12/13)
  * time               (time) datetime64[ns] 2022-03-22T23:01:45.249000072 .....
  * range              (range) float32 87.5 162.4 237.4 ... 8.837e+04 8.844e+04
    azimuth            (time) float32 33.3 33.8 34.3 34.8 ... 31.8 32.3 32.8
    elevation          (time) float32 4.432 4.377 4.322 ... 0.7843 0.8172 0.8502
    latitude           float64 32.81
    longitude          float64 -87.95
    ...                 ...
    x                  (time, range) float32 47.9 88.92 ... 4.786e+04 4.79e+04
    y                  (time, range) float32 72.91 135.4 ... 7.426e+04 7.432e+04
    z                  (time, range) float32 65.0 70.0 ... 1.829e+03 1.83e+03
    gr                 (time, range) float32 87.24 162.0 ... 8.834e+04 8.842e+04
    rays               (time, range) float32 4.432 4.432 4.432 ... 0.8502 0.8502
    bins               (time, range) float32 87.5 162.4 ... 8.837e+04 8.844e+04
Data variables: (12/22)
    NCP                (time, range) float32 ...
    TRIP_FLA           (time, range) float32 ...
    SNRHC              (time, range) float32 ...
    SNRVC              (time, range) float32 ...
    DBZHC              (time, range) float32 ...
    DBZHC_F            (time, range) float32 ...
    ...                 ...
    DBZH               (time, range) float32 ...
    sweep_number       float64 ...
    sweep_fixed_angle  float32 ...
    sweep_mode         |S32 b'azimuth_surveillance'
    follow_mode        |S32 ...
    prt_mode           |S32 ...

In [ ]:
vol

In [ ]:
for cfile in cow1_files:
    ds = 
    display = pyart.graph.RadarMapDisplay(radar)
    fig = plt.figure(figsize=(12, 10), constrained_layout = True)
    ax = plt.subplot(221)
    display.plot_ppi('DBZH', 0, vmin=-10., vmax = 60, cmap = 'pyart_HomeyerRainbow')
    ax = plt.subplot(222)
    display.plot_ppi('VEL_F', 0, vmin=-30., vmax = 30, cmap = 'pyart_NWSVel')
    ax = plt.subplot(223)
    display.plot_ppi('ZDRC', 0, vmin = -7.9, vmax = 7.9, cmap = pyart.graph.cm.RefDiff)
    ax = plt.subplot(224)
    display.plot_ppi('PHIDP', 0, vmin = -179, vmax = 179, cmap = pyart.graph.cm.RefDiff)
    filenames = os.path.join(savedir, cfile.split(os.sep)[-1].split('_to_')[-1].split('.nc')[0] + '.png')
    plt.savefig(filenames, bbox_inches = "tight")
    plt.close()
    del radar
    del display

In [12]:
from PIL import Image
imgs = glob.glob(os.path.join(savedir, '*.png'))
# sorting the files by time
imgs.sort()

images = []
for file_name in imgs:
    images.append(Image.open(file_name))

images[0].save(os.path.join(savedir,'animation_iop1_cow1.gif'), 
               save_all=True, 
               append_images=images[1:], 
               duration=50*1000/len(images), loop=0)

from IPython.display import display
from IPython.display import Image

image1 = Image(filename=os.path.join(savedir,'animation_iop1_cow1.gif'), retina=True)
image2 = Image(filename=os.path.join(savedir,"uf",'uf_iop1_cow1_animation.gif'), retina=True)

display(image1, image2)

In [11]:
uf_files = glob.glob(os.path.join(basedir, radar_type[0], "merged/*nc"))
len(uf_files)

181

In [ ]:
for cfile in uf_files:
    ds = 
    display = pyart.graph.RadarMapDisplay(radar)
    fig = plt.figure(figsize=(12, 10), constrained_layout = True)
    ax = plt.subplot(221)
    display.plot_ppi('DBZHCC_F', 0, vmin=-10., vmax = 60, cmap = 'pyart_HomeyerRainbow')
    ax = plt.subplot(222)
    display.plot_ppi('VEL_F', 0, vmin=-30., vmax = 30, cmap = 'pyart_NWSVel')
    ax = plt.subplot(223)
    display.plot_ppi('ZDRC', 0, vmin = -7.9, vmax = 7.9, cmap = pyart.graph.cm.RefDiff)
    ax = plt.subplot(224)
    display.plot_ppi('PHIDP', 0, vmin = -179, vmax = 179, cmap = pyart.graph.cm.RefDiff)
    filenames = os.path.join(savedir, cfile.split(os.sep)[-1].split('_to_')[-1].split('.nc')[0] + '.png')
    plt.savefig(filenames, bbox_inches = "tight")
    plt.close()
    del radar
    del display

In [11]:
os.path.join(savedir,'animation_iop1_cow1.gif')

'/depot/dawson29/data/Projects/PERiLS/obsdata/2022/Illinois_Mobile_Radar/IOP1/plots/animation_iop1_cow1.gif'

In [ ]:
def filter_radar(radar, vel_field="VEL_F", refl_field="DBZHCC_F", 
                 ncp_field="NCP", rhv_field="RHOHV", phi_field="PHIDP"):
    '''Remove noise based on velocity texture and mask all the fields'''
    # Drop some fields
    fields_to_drop = ["DBMHC", "DBMVC", "VEL", "VS", "VS_F", "VL", "VL_F", "DBZHCC"]
    for field in fields_to_drop:
        if field in radar.fields:
            del radar.fields[field]
        
    texture = pyart.retrieve.calculate_velocity_texture(radar, 
                                                        vel_field=vel_field, 
                                                        wind_size=5, 
                                                        check_nyq_uniform=False)
    radar.add_field('VT',texture,replace_existing=True)
    # create gatefilter
    gf = pyart.filters.GateFilter(radar)
    gf.exclude_invalid(refl_field)
    gf.exclude_outside(refl_field, -20, 90)
    gf.exclude_below("SNRHC", 10)
    gf.exclude_above('VT', 9) # Value found by trial and error for this case
    gf_despeckeld = pyart.correct.despeckle_field(radar, refl_field, gatefilter=gf)
    corr_ZH = radar.fields[refl_field].copy()
    ZH_array = np.ma.masked_where(gf_despeckeld.gate_included == False, radar.fields[refl_field]['data'])
    corr_ZH['data'] = ZH_array
    radar.add_field('DBZH',corr_ZH,replace_existing=True)
    
    # get the mask
    mask = np.ma.getmask(radar.fields['DBZH']['data'])

    # iterate through remaining fields
    for field in radar.fields.keys():
        # mask the field
        radar.fields[field]['data'] = np.ma.masked_where(mask, radar.fields[field]['data'])
    return radar

In [ ]:
def dealiase(radar, vel_name):
    #check to see if radar object has nyquist velocity
    try: 
        gatefilter = pyart.correct.GateFilter(radar)
        corr_vel   = pyart.correct.dealias_region_based(
            radar, vel_field=vel_name, keep_original=False, gatefilter = gatefilter)
        radar.add_field(vel_name, corr_vel, True)
    except:
        None

In [ ]:
def natural_sort_key(s, _re=re.compile(r'(\d+)')):
    return [int(t) if i & 1 else t.lower() for i, t in enumerate(_re.split(s))]

In [ ]:
basedir = "/depot/dawson29/data/Projects/PERiLS/obsdata/2022/Illinois_Mobile_Radar/"
iop1 = os.path.join(basedir,"IOP1")

In [ ]:
cow1 = os.path.join(iop1, "COW1/merged/")
dow7 = os.path.join(iop1, "DOW7/merged/")
dow8 = os.path.join(iop1, "DOW8/merged/")

In [ ]:
cow1_files = sorted(glob.glob(os.path.join(cow1,"*nc")), key=natural_sort_key)
print(f'No. of COW1 files: {len(cow1_files)}')
dow7_files = sorted(glob.glob(os.path.join(dow7,"*nc")), key=natural_sort_key)
print(f'No. of DOW7 files: {len(dow7_files)}')
dow8_files = sorted(glob.glob(os.path.join(dow8,"*nc")), key=natural_sort_key)
print(f'No. of DOW8 files: {len(dow8_files)}')

In [ ]:
out_dir = "/depot/dawson29/data/Projects/PERiLS/obsdata/2022/Illinois_Mobile_Radar/IOP1/"
radar_type = ["COW1", "DOW7", "DOW8"]

In [28]:
import matplotlib.pyplot as plt
from metpy.plots import SkewT
from metpy.calc import potential_temperature

# Initial conditions
temp = 10  # oC
dewpt = 0  # oC
pressure = 850  # mb

# Create the Skew-T plot
fig = plt.figure(figsize=(9, 9))
skew = SkewT(fig, rotation=45)

# Plot the location of the temperature and dewpoint with red and green dots, respectively
skew.plot(pressure, temp, 'ro', label='Temperature')
skew.plot(pressure, dewpt, 'go', label='Dewpoint')

# Annotate the potential temperature of the parcel in blue
pot_temp = potential_temperature(pressure, temp)
skew.plot(pressure, pot_temp, 'bo', label='Potential Temperature')

# Draw the path of the parcel's temperature and dewpoint as it is lifted dry adiabatically to saturation
skew.plot_dry_adiabats()
skew.plot_moist_adiabats()
lcl_pressure, lcl_temp = skew.lift(pressure, temp, dewpt)
skew.plot(lcl_pressure, lcl_temp, 'ko', label='LCL')

# Draw the path of the parcel's temperature as it is lifted moist adiabatically to a pressure of 300 mb
temp_300mb, _ = skew.find_intersection(skew.moist_adiabats, 300)
skew.plot(300, temp_300mb, 'ko')

# Annotate the parcel's temperature and dewpoint at this level
temp_dewpt_300mb = skew.dewpoint_from_mixing_ratio(skew.mixing_ratio_from_pressure_and_dewpt(300, dewpt))
skew.plot(300, temp_dewpt_300mb, 'ko')

# Add labels and legend
skew.plot_mixing_lines()
skew.ax.set_ylim(-100, 100)
skew.ax.set_xlim(1000, 100)
skew.ax.legend(loc='upper left')
plt.show()

ModuleNotFoundError: No module named 'metpy'

In [29]:
!conda install -c conda-forge metpy --yes

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::jupyterlab==3.2.1=pyhd3eb1b0_1
  - defaults/noarch::jupyterlab_server==2.8.2=pyhd3eb1b0_0
  - defaults/linux-64::anaconda==2021.11=py39_0
  - defaults/linux-64::_ipyw_jlab_nb_ext_conf==0.1.0=py39h06a4308_0
done


==> WARNING: A newer version of conda exists. <==
  current version: 4.11.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /apps/cent7/jupyterhub

  added / updated specs:
    - metpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2022.10  |           py39_2          67 KB
    anaconda-custom            |           py39_1           3 KB
    automat-22.10.0            |     pyhd8ed1ab_0  

libidn2-2.3.4        | 157 KB    | ##################################### | 100% 
proj-8.0.1           | 3.1 MB    | ##################################### | 100% 
libunistring-0.9.10  | 1.4 MB    | ##################################### | 100% 
boto3-1.26.55        | 75 KB     | ##################################### | 100% 
hvplot-0.8.2         | 3.0 MB    | ##################################### | 100% 
metpy-1.4.0          | 316 KB    | ##################################### | 100% 
itemadapter-0.7.0    | 14 KB     | ##################################### | 100% 
hyperlink-21.0.0     | 71 KB     | ##################################### | 100% 
requests-file-1.5.1  | 7 KB      | ##################################### | 100% 
scrapy-2.7.1         | 825 KB    | ##################################### | 100% 
conda-4.14.0         | 1011 KB   | ##################################### | 100% 
_anaconda_depends-20 | 67 KB     | ##################################### | 100% 
datashader-0.14.3    | 16.3 